In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [2]:
import pandas as pd

# Define the common path to the files
data_path = '/kaggle/input/m5-forecasting-accuracy/'

# Load datasets using the common path
sales_train_validation = pd.read_csv(f'{data_path}sales_train_validation.csv')
sales_train_evaluation = pd.read_csv(f'{data_path}sales_train_evaluation.csv')
calendar = pd.read_csv(f'{data_path}calendar.csv')
sell_prices = pd.read_csv(f'{data_path}sell_prices.csv')
sample_submission = pd.read_csv(f'{data_path}sample_submission.csv')

# Display the first few rows of each DataFrame
print("Sales Train Validation Data:")
print(sales_train_validation.head())

print("\nSales Train Evaluation Data:")
print(sales_train_evaluation.head())

print("\nCalendar Data:")
print(calendar.head())

print("\nSell Prices Data:")
print(sell_prices.head())

print("\nSample Submission Data:")
print(sample_submission.head())

# Display data types and check for missing values
print("\nSales Train Validation Data Info:")
print(sales_train_validation.info())

print("\nSales Train Evaluation Data Info:")
print(sales_train_evaluation.info())

print("\nCalendar Data Info:")
print(calendar.info())

print("\nSell Prices Data Info:")
print(sell_prices.info())

Sales Train Validation Data:
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  \
0       CA    0    0    0    0  ...       1       3       0       1       1   
1       CA    0    0    0    0  ...       0       0       0       0       0   
2       CA    0    0    0    0  ...       2       1       2       1       1   
3       CA    0    0    0    0  ...       1       0       5       4       1   
4       CA    0    0    0    0  ...       2       1       1       0       1   

   d_1909  d_19

In [3]:
# Melt the sales_train_validation data
sales_train_validation_melted = sales_train_validation.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Display the first few rows of the melted DataFrame
print("Melted Sales Train Validation Data:")
print(sales_train_validation_melted.head())

Melted Sales Train Validation Data:
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales  
0       CA  d_1      0  
1       CA  d_1      0  
2       CA  d_1      0  
3       CA  d_1      0  
4       CA  d_1      0  


In [4]:
def merge_in_chunks(df, calendar, prices, chunk_size=1500000):
    # List to hold the merged chunks
    merged_chunks = []
    
    # Number of chunks
    num_chunks = len(df) // chunk_size + 1
    
    for i in range(num_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        chunk = df[start:end]
        chunk = chunk.merge(calendar, on='d', how='left')
        chunk = chunk.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
              
        merged_chunks.append(chunk)
        
        print(f"Chunk {i+1}/{num_chunks} merged")
    
    # Concatenate all chunks
    merged_df = pd.concat(merged_chunks, ignore_index=True)
    
    return merged_df

# Merge the melted DataFrame in chunks
sales_train_validation_merged = merge_in_chunks(sales_train_validation_melted, calendar, sell_prices)

# Display the first few rows of the merged DataFrame
print("Merged Sales Train Validation Data:")
print(sales_train_validation_merged.head())

Chunk 1/39 merged
Chunk 2/39 merged
Chunk 3/39 merged
Chunk 4/39 merged
Chunk 5/39 merged
Chunk 6/39 merged
Chunk 7/39 merged
Chunk 8/39 merged
Chunk 9/39 merged
Chunk 10/39 merged
Chunk 11/39 merged
Chunk 12/39 merged
Chunk 13/39 merged
Chunk 14/39 merged
Chunk 15/39 merged
Chunk 16/39 merged
Chunk 17/39 merged
Chunk 18/39 merged
Chunk 19/39 merged
Chunk 20/39 merged
Chunk 21/39 merged
Chunk 22/39 merged
Chunk 23/39 merged
Chunk 24/39 merged
Chunk 25/39 merged
Chunk 26/39 merged
Chunk 27/39 merged
Chunk 28/39 merged
Chunk 29/39 merged
Chunk 30/39 merged
Chunk 31/39 merged
Chunk 32/39 merged
Chunk 33/39 merged
Chunk 34/39 merged
Chunk 35/39 merged
Chunk 36/39 merged
Chunk 37/39 merged
Chunk 38/39 merged
Chunk 39/39 merged
Merged Sales Train Validation Data:
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     

In [5]:
# Fill missing sell prices with the last available price
sales_train_validation_merged['sell_price'] = sales_train_validation_merged['sell_price'].ffill()

# Fill missing event data with 'None'
event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
sales_train_validation_merged[event_cols] = sales_train_validation_merged[event_cols].fillna('None')

In [6]:
def create_rolling_features(df, window_sizes):
    for window in window_sizes:
        df[f'rolling_mean_{window}'] = df.groupby(['id'])['sales'].shift(1).rolling(window=window).mean()
        df[f'rolling_std_{window}'] = df.groupby(['id'])['sales'].shift(1).rolling(window=window).std()
    return df

# Apply rolling features
window_sizes = [7, 30]
sales_train_validation_merged = create_rolling_features(sales_train_validation_merged, window_sizes)

In [7]:
# Handle NaNs in rolling features
rolling_mean_cols = [f'rolling_mean_{window}' for window in window_sizes]
rolling_std_cols = [f'rolling_std_{window}' for window in window_sizes]

# Fill NaNs in rolling features with the overall mean and std (assuming that initial periods can use overall stats)
sales_train_validation_merged[rolling_mean_cols] = sales_train_validation_merged[rolling_mean_cols].fillna(sales_train_validation_merged['sales'].mean())
sales_train_validation_merged[rolling_std_cols] = sales_train_validation_merged[rolling_std_cols].fillna(sales_train_validation_merged['sales'].std())

In [8]:
#Select and identify departments for splitting.
items_location = sales_train_validation_merged['id'].unique()
items =items_location[0]
items_data = sales_train_validation_merged[sales_train_validation_merged['id'] == items]

# Display the first few rows of the filtered data
print(items_data.head())

                                   id        item_id    dept_id   cat_id  \
0       HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
30490   HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
60980   HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
91470   HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
121960  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   

       store_id state_id    d  sales        date  wm_yr_wk  ... event_name_2  \
0          CA_1       CA  d_1      0  2011-01-29     11101  ...         None   
30490      CA_1       CA  d_2      0  2011-01-30     11101  ...         None   
60980      CA_1       CA  d_3      0  2011-01-31     11101  ...         None   
91470      CA_1       CA  d_4      0  2011-02-01     11101  ...         None   
121960     CA_1       CA  d_5      0  2011-02-02     11101  ...         None   

        event_type_2  snap_CA  snap_TX snap_WI sell_price roll

In [9]:
# Define the features and target
features = ['sell_price', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7', 'rolling_std_30']
target = 'sales'

# Split features and target
X_items=items_data [features]
y_items=items_data[target]

# Display the first few rows of features and target
print(X_items.head())
print(y_items.head())

        sell_price  wm_yr_wk  snap_CA  snap_TX  snap_WI  rolling_mean_7  \
0              NaN     11101        0        0        0        1.126322   
30490          4.0     11101        0        0        0        1.126322   
60980          4.0     11101        0        0        0        0.571429   
91470          4.0     11101        1        1        0        1.428571   
121960         4.0     11101        1        0        1        0.571429   

        rolling_mean_30  rolling_std_7  rolling_std_30  
0              1.126322       3.873108        3.873108  
30490          1.126322       3.873108        3.873108  
60980          2.666667       1.511858        7.326538  
91470          3.066667       2.507133        7.619998  
121960         2.133333       0.975900        6.420513  
0         0
30490     0
60980     0
91470     0
121960    0
Name: sales, dtype: int64


In [10]:
from sklearn.model_selection import train_test_split

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_items, y_items, test_size=0.2, random_state=42)

# Display the shapes of the training and validation sets
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(1530, 9) (383, 9)
(1530,) (383,)


In [11]:
# Standardize the data before training the model.
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Display the first few rows of the scaled training data
print(pd.DataFrame(X_train_scaled, columns=features).head())

   sell_price  wm_yr_wk   snap_CA   snap_TX   snap_WI  rolling_mean_7  \
0    0.920376  1.088581 -0.698804 -0.699840 -0.698804       -1.273441   
1    0.920376  1.095260 -0.698804 -0.699840 -0.698804       -0.088847   
2    0.920376  1.382457 -0.698804 -0.699840 -0.698804        0.384990   
3   -1.018336 -1.275785  1.431017  1.428898  1.431017       -0.562685   
4   -1.018336 -0.607885  1.431017 -0.699840  1.431017       -0.325766   

   rolling_mean_30  rolling_std_7  rolling_std_30  
0        -1.565165      -1.071329       -1.268703  
1        -0.664241      -0.607328       -0.823632  
2        -0.423995      -0.326982       -0.412528  
3        -0.904487      -0.539596       -0.350522  
4        -0.454026       0.273138       -0.583517  


In [12]:
# Fill NaNs in X_train_scaled with the mean of the respective columns
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features)
X_train_scaled.fillna(X_train_scaled.mean(), inplace=True)

# Fill NaNs in X_val_scaled with the mean of the respective columns
X_val_scaled = pd.DataFrame(X_val_scaled, columns=features)
X_val_scaled.fillna(X_val_scaled.mean(), inplace=True)

# Convert back to numpy arrays
X_train_scaled = X_train_scaled.values
X_val_scaled = X_val_scaled.values

# Check for NaNs again
print("NaNs in X_train_scaled after filling:", np.isnan(X_train_scaled).sum())
print("NaNs in X_val_scaled after filling:", np.isnan(X_val_scaled).sum())

NaNs in X_train_scaled after filling: 0
NaNs in X_val_scaled after filling: 0


In [13]:
from sklearn.linear_model import SGDRegressor

# Initialize the model
model = SGDRegressor(max_iter=1000, tol=1e-3)

# Train the model on the entire dataset
model.fit(X_train_scaled, y_train)

# Save the model and scaler for the current department
models = {}
models[items] = (model, scaler)

# Check the model coefficients
print(model.coef_)

[ 1.19006376e-01  1.60868778e-01  1.62525205e-02 -1.63111745e-03
  1.45277307e-04  3.86473758e-02 -4.99664304e-02 -6.02781471e-02
  3.73706985e-02]


In [14]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model.predict(X_val_scaled)

# Calculate and print the mean squared error
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error for items {items}: {mse}")

Mean Squared Error for items HOBBIES_1_001_CA_1_validation: 0.3554397322625584


In [ ]:
models = {}  # Dictionary to store models and scalers for each department
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Get unique ids
id = sales_train_validation_merged['id'].unique()

for items in id:
    # Filter data for the current department
    items_data = sales_train_validation_merged[sales_train_validation_merged['id'] == items]
    
    # Prepare features and target
    X = items_data[features]
    y = items_data[target]
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Handle NaNs
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=features)
    X_train_scaled.fillna(X_train_scaled.mean(), inplace=True)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=features)
    X_val_scaled.fillna(X_val_scaled.mean(), inplace=True)
    X_train_scaled = X_train_scaled.values
    X_val_scaled = X_val_scaled.values
    
    # Initialize the model
    model = SGDRegressor(max_iter=1000, tol=1e-3)
    
    # Train the model on the entire dataset
    model.fit(X_train_scaled, y_train)
    
    # Store the model and scaler for the current department
    models[items] = (model, scaler)
    
    # Evaluate the model
    y_pred = model.predict(X_val_scaled)
    mse = mean_squared_error(y_val, y_pred)
    print(f"Mean Squared Error for items {items}: {mse}")

print("All items processed and models stored.")

Mean Squared Error for items HOBBIES_1_001_CA_1_validation: 0.35481585201983046
Mean Squared Error for items HOBBIES_1_002_CA_1_validation: 0.30918468728559767
Mean Squared Error for items HOBBIES_1_003_CA_1_validation: 0.16036616669838907
Mean Squared Error for items HOBBIES_1_004_CA_1_validation: 3.420307889178062
Mean Squared Error for items HOBBIES_1_005_CA_1_validation: 1.5478990212935815
Mean Squared Error for items HOBBIES_1_006_CA_1_validation: 2.3588597352970195
Mean Squared Error for items HOBBIES_1_007_CA_1_validation: 0.22940930980359037
Mean Squared Error for items HOBBIES_1_008_CA_1_validation: 84.83509069658285
Mean Squared Error for items HOBBIES_1_009_CA_1_validation: 3.060109129413271
Mean Squared Error for items HOBBIES_1_010_CA_1_validation: 0.8685135196003236
Mean Squared Error for items HOBBIES_1_011_CA_1_validation: 0.06819054955377457
Mean Squared Error for items HOBBIES_1_012_CA_1_validation: 0.4282342187892799
Mean Squared Error for items HOBBIES_1_013_CA_1_va